In [1]:
from itertools import combinations

import matplotlib.pyplot as plt
import numpy as np

import weight_parser as wp
import stats as s
import constants as C
import plotting as p
import metadata as m
import utils as u

In [16]:
sim_dict = dict()
for xai_method in C.XAI_METHODS[-1:]:
    sims, mats = [], []
    for model_name in C.MODELS:
        if xai_method == "ig" and model_name == "convnext-base":
            continue
        seeds = [42, 43, 44] if model_name != "convnext-base" else [42, 43]
        mats.extend([wp.Weights(model_name=model_name, seed=seed, xai_method=xai_method, verbose=False).normalize_df()\
                     for seed in seeds])

    print(f"XAI Method: {xai_method.capitalize()}", end=" ")
    loop = list(combinations(mats, r=2))
    print(f"# combinations: {len(loop)}")
    for mat1, mat2 in loop:
        simmat = mat1 @ mat2.T
        sim = np.diag(simmat).mean()
        sims.append(sim)
    sim_dict[xai_method] = sims

XAI Method: Ig # combinations: 66


In [17]:
import json

with open("./similarity.json", mode="r") as f:
    _sim_dict = json.load(fp=f)

In [18]:
sim_dict.update(_sim_dict)

In [20]:
import numpy as np
for xai_method in sim_dict:
    print(f"XAI {xai_method:11s}: {np.mean(sim_dict[xai_method]):.4f}")

XAI ig         : 0.4991
XAI gradxinput : 0.5031
XAI gcam_avg   : 0.1388
XAI gbp        : 0.3261
XAI ggcam      : 0.0940
XAI ggcam_avg  : 0.2649
XAI deeplift   : 0.5029
